In [6]:
from tqdm import tqdm
import open_clip

In [7]:
model, _, preprocess = open_clip.create_model_and_transforms(model_name='ViT-L-14', pretrained='openai')

In [8]:
model.eval().to('cpu')

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1024,), eps=1e-05, elementwise_affine=Tru

#### 创建pandas

In [9]:
import pandas as pd
import json
from PIL import Image
import torch
import numpy as np

In [10]:
json_data = json.load(open('/Users/chenchaofan/python_project/data/VisA/meta.json'))

In [11]:
json_data

{'train': {'candle': [{'img_path': 'candle/Data/Images/Normal/0836.JPG',
    'mask_path': '',
    'cls_name': 'candle',
    'specie_name': '',
    'anomaly': 0},
   {'img_path': 'candle/Data/Images/Normal/0451.JPG',
    'mask_path': '',
    'cls_name': 'candle',
    'specie_name': '',
    'anomaly': 0},
   {'img_path': 'candle/Data/Images/Normal/0516.JPG',
    'mask_path': '',
    'cls_name': 'candle',
    'specie_name': '',
    'anomaly': 0},
   {'img_path': 'candle/Data/Images/Normal/0977.JPG',
    'mask_path': '',
    'cls_name': 'candle',
    'specie_name': '',
    'anomaly': 0},
   {'img_path': 'candle/Data/Images/Normal/0249.JPG',
    'mask_path': '',
    'cls_name': 'candle',
    'specie_name': '',
    'anomaly': 0},
   {'img_path': 'candle/Data/Images/Normal/0548.JPG',
    'mask_path': '',
    'cls_name': 'candle',
    'specie_name': '',
    'anomaly': 0},
   {'img_path': 'candle/Data/Images/Normal/0362.JPG',
    'mask_path': '',
    'cls_name': 'candle',
    'specie_name': '',

In [12]:
len(json_data['train']['cashew'])

450

In [13]:
len(json_data['test']['cashew'])

150

In [14]:
cachew_data = json_data['train']['cashew'] + json_data['test']['cashew']

In [15]:
len(cachew_data)

600

In [16]:
df = pd.DataFrame(cachew_data)

In [17]:
df.head()

,img_path,mask_path,cls_name,specie_name,anomaly
0,cashew/Data/Images/Normal/062.JPG,,cashew,,0
1,cashew/Data/Images/Normal/253.JPG,,cashew,,0
2,cashew/Data/Images/Normal/393.JPG,,cashew,,0
3,cashew/Data/Images/Normal/408.JPG,,cashew,,0
4,cashew/Data/Images/Normal/426.JPG,,cashew,,0


In [18]:
df.to_csv('./cashew_data.csv', index=False)

#### 计算每张图像的语义特征

In [19]:
encoding_array = []
img_path_list = []

for img_path in tqdm(df['img_path']):
    img_path = '/Users/chenchaofan/python_project/data/VisA/'+img_path
    img_path_list.append(img_path)
    img_pil = Image.open(img_path).convert("RGB")
    input_img = preprocess(img_pil).unsqueeze(0).to('cpu')
    with torch.no_grad():
        image_features = model.encode_image(input_img).detach().squeeze().cpu().numpy()
    encoding_array.append(image_features)
    # print(image_features.shape)
    

100%|██████████| 600/600 [03:46<00:00,  2.65it/s]


In [23]:
np.save('cashew_visual_features.npy', encoding_array)